# SST modes vs. GMST regression

1. regress modes of SST variability against GMST variability

In [ ]:
import sys
import scipy as sp
import numpy as np
import xarray as xr
import pandas as pd
import statsmodels.api as sm
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
matplotlib.rc_file('../rc_file_paper')
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
sys.path.append("..")
from paths import path_prace, path_results
from constants import spy
from filters import lowpass
from itertools import combinations
from xr_regression import xr_quadtrend
from bb_analysis_timeseries import AnalyzeTimeSeries as ATS

## time series into dataframe

In [ ]:
# GMST
gmst_had  = xr.open_dataarray(f'{path_prace}/GMST/GMST_dt_yrly_had.nc', decode_times=False)
gmst_had  = gmst_had.isel({'time':slice(9,158)})
gmst_ctrl = xr.open_dataset(f'{path_prace}/GMST/GMST_ctrl.nc').GMST.isel({'time':slice(50,300)})
gmst_ctrl['time'] = (gmst_ctrl.time/365).astype(dtype=int)
gmst_ctrl -= xr_quadtrend(gmst_ctrl)
gmst_lpd  = xr.open_dataset(f'{path_prace}/GMST/GMST_lpd.nc').GMST.isel({'time':slice(0,250)})
gmst_lpd['time'] = (gmst_lpd.time/365).astype(dtype=int)
gmst_lpd -= xr_quadtrend(gmst_lpd)

# OHC
ctrl_qd = xr.open_dataset(f'{path_prace}/OHC/OHC_integrals_ctrl_qd.nc', decode_times=False)
lpd_qd  = xr.open_dataset(f'{path_prace}/OHC/OHC_integrals_lpd_qd.nc' , decode_times=False)

TOA_ctrl = xr.open_dataarray(f'{path_prace}/TOA/TOM_ctrl.nc', decode_times=False).isel(time=slice(50,300))
TOA_lpd  = xr.open_dataarray(f'{path_prace}/TOA/TOM_lpd.nc' , decode_times=False).isel(time=slice(0,250))

SHF_ctrl = xr.open_dataset(f'{path_prace}/OHC/SHF_ctrl.nc', decode_times=False).isel(time=slice(50,300))
SHF_lpd  = xr.open_dataset(f'{path_prace}/OHC/SHF_lpd.nc' , decode_times=False).isel(time=slice(0,250))

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(9,3), sharey='row', sharex='col', gridspec_kw={"width_ratios":[149, 250, 250]})
for i in range(3):
    ax[i].axhline(0, c='grey', lw=.5)
gmst_had .plot(ax=ax[0])
gmst_ctrl[:-1].plot(ax=ax[1])
gmst_lpd .plot(ax=ax[2])

lowpass(gmst_had ,13).plot(ax=ax[0])
lowpass(gmst_ctrl[:-1],13).plot(ax=ax[1])
lowpass(gmst_lpd ,13).plot(ax=ax[2])

In [ ]:
# TOA
plt.figure(figsize=(8,5))
for i, ts in enumerate([TOA_ctrl, TOA_lpd]):
    (spec, freq, jackknife) = ATS(ts).spectrum()
    plt.plot(freq, spec, label=['HIGH', 'LOW'][i], ls=['-', '--'][i])
    plt.legend(fontsize=6)
plt.ylabel('TOA spectral power')
# plt.semilogx()
plt.loglog()

In [ ]:
# GMST
plt.figure(figsize=(8,5))
plt.gca().tick_params(labelsize=14)
for i in np.arange(0,101,10):
    (spec, freq, jackknife) = ATS(gmst_ctrl.isel({'time':slice(i,149+i)})).spectrum()
    plt.plot(freq, spec, c='C1', lw=.5, ls='--', alpha=.5)
    (spec, freq, jackknife) = ATS(gmst_lpd.isel({'time':slice(i,149+i)})).spectrum()
    plt.plot(freq, spec, c='C2', lw=.5, ls='--', alpha=.5)

for i, ts in enumerate([gmst_had, gmst_ctrl, gmst_lpd]):
    (spec, freq, jackknife) = ATS(ts).spectrum()
    plt.plot(freq, spec, label=['HIST', 'HIGH', 'LOW'][i])
    plt.legend(fontsize=14)
    
plt.xlabel(r'frequency [year$^{-1}$]', fontsize=14)
plt.ylabel('GMST spectral power', fontsize=14)
plt.loglog()
plt.savefig(f'{path_results}/Battisti/GMST_spectra')

# \Delta GMST
plt.figure(figsize=(8,5))
plt.gca().tick_params(labelsize=14)
for i in np.arange(0,101,10):
    ts1 = gmst_ctrl.isel({'time':slice(i,149+i)})
    ts2 = gmst_lpd .isel({'time':slice(i,149+i)})
    for j, ts in enumerate([ts1, ts2]):
        ts_ = (ts-ts.shift(time=1)).dropna(dim='time')
        (spec, freq, jackknife) = ATS(ts_).spectrum()
        plt.plot(freq, spec, c=f'C{j+1}', lw=.5, ls='--', alpha=.5)

for i, ts in enumerate([gmst_had, gmst_ctrl, gmst_lpd]):
    (spec, freq, jackknife) = ATS((ts-ts.shift(time=1)).dropna(dim='time')).spectrum()
    plt.plot(freq, spec, label=['HIST', 'HIGH', 'LOW'][i])
    plt.legend(fontsize=14)
    
plt.xlabel(r'frequency [year$^{-1}$]', fontsize=14)
plt.ylabel(r'$\Delta$GMST spectral power', fontsize=14)
# plt.semilogx()
plt.loglog()

In [ ]:
plt.figure(figsize=(8,5))
plt.gca().tick_params(labelsize=14)
for i, ts in enumerate([ctrl_qd, lpd_qd]):
    for j, basin in enumerate(['Global', 'Atlantic', 'Pacific', 'Southern']):
        c = ['k' ,'C0','C1','C3'][j]
        (spec, freq, jackknife) = ATS(ts[f'OHC_{basin}_Ocean']).spectrum()
        plt.plot(freq, spec, label=[f'HIGH {basin}', f'LOW {basin}'][i], ls=['-', '--'][i], c=c, alpha=[1,.7,.7,.7][j])
        plt.legend(fontsize=6)
plt.ylabel('OHC spectral power')
plt.loglog()
# plt.semilogx()

In [ ]:
plt.figure(figsize=(8,5))
plt.gca().tick_params(labelsize=14)
for i, ts in enumerate([ctrl_qd, lpd_qd]):
    for j, basin in enumerate([ 'Global', 'Atlantic', 'Pacific', 'Southern']):
        c = ['k' ,'C0','C1','C3'][j]
        ts_ = (ts[f'OHC_{basin}_Ocean'] - ts[f'OHC_{basin}_Ocean'].shift(time=1)).dropna(dim='time')
        (spec, freq, jackknife) = ATS(ts_).spectrum()
        plt.plot(freq, spec*10**-j, label=[f'HIGH {basin}', f'LOW {basin}'][i], ls=['-', '--'][i], c=c, alpha=[1,.5,.5,1.5][j])
        plt.legend(fontsize=6)
plt.ylabel(r'$\Delta$ OHC spectral power', fontsize=14)
plt.loglog()
# plt.semilogx()

In [ ]:
plt.figure(figsize=(8,5))
plt.gca().tick_params(labelsize=14)

for i, ts in enumerate([SHF_ctrl, SHF_lpd]):
    for j, basin in enumerate(['Global', 'Pacific', 'Atlantic', 'Southern']):
        c = ['k' ,'C1','C0','C3'][j]
        ts_ = (ts[f'{basin}_Ocean'] - xr_quadtrend(ts[f'{basin}_Ocean']))/1e21
        (spec, freq, jackknife) = ATS(ts_).spectrum()
        plt.plot(freq, spec*10**-j, label=[f'HIGH {basin}', f'LOW {basin}'][i], ls=['-', '--'][i], c=c, alpha=[1,.7,.7,.7][j])
plt.legend(fontsize=12)
plt.xlabel(r'frequency [year$^{-1}$]', fontsize=14)
plt.ylabel('SHF spectral power', fontsize=14)
# plt.semilogx()
plt.loglog()
plt.savefig(f'{path_results}/Battisti/SHF_spectra')

In [ ]:
f, ax = plt.subplots(3,1, figsize=(8,8), sharex=True)
for i in range(3):
    ax[i].loglog()
    ax[i].tick_params(labelsize=12)

    
for i in range(3):
    gmst = [gmst_had, gmst_ctrl, gmst_lpd][i]
    (spec, freq, jackknife) = ATS(gmst).spectrum()
    ax[0].plot(freq, spec, label=['HIST', 'HIGH', 'LOW'][i])
    ax[0].legend(fontsize=14)
    
for i in range(2):
    toa = [TOA_ctrl, TOA_lpd][i]
    toa_ = (toa-xr_quadtrend(toa))*spy/1e21
    (spec, freq, jackknife) = ATS(toa_).spectrum()
    ax[1].plot(freq, spec, label=f'TOA {["HIGH", "LOW"][i]}', c=f'C{i+1}')
    ax[1].legend()

    shf  = [SHF_ctrl, SHF_lpd][i]
    shf_ = (shf[f'Global_Ocean'] - xr_quadtrend(shf[f'Global_Ocean']))/1e21
    (spec, freq, jackknife) = ATS(shf_).spectrum()
    ax[1].plot(freq, spec, c=f'C{i+1}', ls='--', label=f'SHF {["HIGH", "LOW"][i]}')
    ax[1].legend()
    
    toa_['time'] = shf_.time
    div = toa_-shf_
    (spec, freq, jackknife) = ATS(div).spectrum()
    ax[2].plot(freq, spec, c=f'C{i+1}', label=f'TOA-SHF {["HIGH", "LOW"][i]}')
    ax[2].legend()

ax[0].set_ylabel('GMST spectral power', fontsize=14)
ax[1].set_ylabel('TOA & SHF spectral power', fontsize=14)
ax[2].set_ylabel('TOA-SHF spectral power', fontsize=14)

ax[-1].set_xlabel(r'frequency [year$^{-1}$]', fontsize=14)
plt.align_ylabels()
plt.savefig(f'{path_results}/Battisti/TOA_GMST_SHF_spectra')

In [ ]:
for i in range(10):
    plt.plot([0,1], [i,i], c=f'C{i}')